In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [2]:
x,Y = load_iris(return_X_y=True,as_frame = True)

###Perceptron MULTICAMADAS

In [3]:
x0 = np.ones(len(x))*-1

x0_colum = pd.DataFrame(x0)

X = pd.concat([x0_colum,x],axis=1,ignore_index=True)

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=32)

In [4]:
from keras.utils import to_categorical

# Suponha que y_train contenha os rótulos 0, 1 e 2
y_train_one_hot = to_categorical(y_train, num_classes=3)
y_train_one_hot

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0

###Iniciando com valores aletórios as matrizes de pesos


In [23]:
np.random.seed(39)
w1 = np.random.uniform(low=-1, high=1, size=(3, 5))
w2 = np.random.uniform(low=-1, high=1, size=(2, 4))
w3 = np.random.uniform(low=-1, high=1, size=(3, 3))

In [22]:
n = 0.5
e = 1e-6
epoca = 0

###Função de Ativação

In [7]:
def g(u):
    return np.maximum(0, u)


def g_dev(u):
  if u > 0:
    return 1

  else:
    return 0

In [8]:
def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x)

def softmax_derivative(x):
    s = softmax(x)
    return s * (1 - s)


Derivadas parciais da função softmax em relação a x: [0.23557444 0.23557444]


In [9]:
def erm(d,y,X_tr,w1,w2,w3):
  erro = 0
  for i in range(len(X_tr)):
    i_1,y_1,i_2,y_2,i_3,y_3 = forward(X_tr,w1,w2,w3,i)
    erro += ((np.argmax(d[i]) - np.argmax(y_3))**2)/2


  return erro/len(X_tr)




In [14]:
erm(y_train_one_hot,y_3,X_train_norm,w1,w2,w3)

0.8083333333333333

In [10]:
def forward(X,w1,w2,w3,i):
  i_1 = []
  for j in range(len(w1)):
    z = np.dot(X_train.iloc[i,:],w1[j].T)
    i_1.append(z)

  y_1 = g(i_1)
  y_1 = np.insert(y_1,0,-1)

  i_2 = []
  for j in range(len(w2)):
    w = np.dot(y_1,w2[j].T)
    i_2.append(w)

  y_2 = g(i_2)
  y_2 = np.insert(y_2,0,-1)

  i_3 = []
  for j in range(len(w3)):
    k = np.dot(y_2, w3[j].T)
    i_3.append(k)

  y_3 = softmax(i_3)

  return i_1,y_1,i_2,y_2,i_3,y_3





In [11]:
def normalization(data):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    return data

In [12]:
X_train_norm = pd.DataFrame(normalization(X_train))

In [19]:
i_1,y_1,i_2,y_2,i_3,y_3 = forward(X_train_norm,w1,w2,w3,0)

In [ ]:
a = softmax_derivative(i_3)

0.22152843071032843

###Não está atualizando os pesos w1,w2 e w3


In [24]:
while True:
  errom = erm(y_train_one_hot,y_3,X_train_norm,w1,w2,w3)
  for w in range(1,len(X_train_norm) - 1 ):
    i_1,y_1,i_2,y_2,i_3,y_3 = forward(X_train_norm,w1,w2,w3,w)

    ##Passo backward
    delta_3 = []
    deriv_i_3 = softmax_derivative(i_3)
    for i in range(len(w3)):
      delta = (y_train_one_hot[w,i] - y_3[i]) * deriv_i_3[i]

      delta_3.append(delta)


    for j in range(len(w3)):
      for i in range(len(w3[j])):
        w3[j][i] +=  n*(delta_3[j] * y_2[j])






    ####
    delta_2 = []
    deriv_i_2 = softmax_derivative(i_2)
    for i in range(len(w2)):
      z = 0
      for k in range(len(w3)):
        z += np.dot(delta_3,w3[k])


      delta_2.append(z * deriv_i_2[i])


    for j in range(len(w2)):
      for i in range(len(w2[j])):
        w2[j][i] += n*(delta_2[j] * y_1[j])



    ######
    delta_1 = []
    deriv_i_1 = softmax_derivative(i_1)
    for i in range(len(w1)):
      z = 0
      for k in range(len(w2)):
        z+= np.dot(delta_2,w2.T[k])

      delta_1.append(z * deriv_i_1[i])



    for j in range(len(w1)):
      for i in range(len(w1[j])):
        w1[j][i] += n*(delta_1[j] * X_train_norm.iloc[w,i])






  em_atual = erm(y_train_one_hot,y_3,X_train_norm,w1,w2,w3)
  print(em_atual)
  print(errom)
  epoca +=1


  if abs(em_atual-errom) < e:
    break



0.5541666666666667
1.4875
0.31666666666666665
0.5541666666666667
0.8333333333333334
0.31666666666666665
0.8083333333333333
0.8333333333333334
0.8041666666666667
0.8083333333333333
0.8041666666666667
0.8041666666666667


In [25]:
X_test_norm = pd.DataFrame(normalization(X_test))
for i in range(len(X_test_norm)):
  i_1,y_1,i_2,y_2,i_3,y_3 = forward(X_test_norm,w1,w2,w3,i)
  print(y_3)


[0.34589167 0.00238813 0.65172021]
[3.40616594e-01 2.83659290e-04 6.59099747e-01]
[0.3444704  0.00125718 0.65427242]
[0.34619453 0.00279342 0.65101204]
[0.34713708 0.00530322 0.6475597 ]
[3.31680359e-01 1.11034184e-05 6.68308538e-01]
[3.29600653e-01 5.22696902e-06 6.70394120e-01]
[0.34680994 0.01424714 0.63894292]
[0.3473119  0.00668941 0.64599869]
[0.34701466 0.00473828 0.64824705]
[3.22375765e-01 3.74960862e-07 6.77623860e-01]
[0.34641567 0.00315897 0.65042536]
[3.25311168e-01 1.09753812e-06 6.74687735e-01]
[0.34685362 0.00417945 0.64896693]
[3.29186787e-01 4.49813192e-06 6.70808715e-01]
[3.41812981e-01 4.42897888e-04 6.57744121e-01]
[0.34724429 0.00600177 0.64675393]
[0.34498626 0.00156772 0.65344602]
[0.34699633 0.00466624 0.64833743]
[3.34882821e-01 3.53244659e-05 6.65081854e-01]
[0.34716108 0.011546   0.64129292]
[0.3462893  0.00294149 0.65076922]
[3.32810582e-01 1.67096302e-05 6.67172709e-01]
[0.29852697 0.16936494 0.53210809]
[0.34548738 0.02122175 0.63329087]
[3.33928607e-01 2

In [ ]:
def normalization(data):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    data = scaler.fit_transform(data)

    return data

In [ ]:
X_train_norm = pd.DataFrame(normalization(X_train))
X_train

,0,1,2,3,4
79,-1.0,5.7,2.6,3.5,1.0
38,-1.0,4.4,3.0,1.3,0.2
41,-1.0,4.5,2.3,1.3,0.3
144,-1.0,6.7,3.3,5.7,2.5
98,-1.0,5.1,2.5,3.0,1.1
...,...,...,...,...,...
88,-1.0,5.6,3.0,4.1,1.3
124,-1.0,6.7,3.3,5.7,2.1
54,-1.0,6.5,2.8,4.6,1.5
133,-1.0,6.3,2.8,5.1,1.5


###Pecpetron com biblotecas

In [ ]:
import tensorflow as tf

In [ ]:
classifier = tf.keras.models.Sequential()

In [ ]:
classifier.add(tf.keras.layers.Dense(activation="relu",input_dim= 5,units = 3, kernel_initializer = 'uniform'))
classifier.add(tf.keras.layers.Dense(activation="relu",units = 2, kernel_initializer = 'uniform'))

classifier.add(tf.keras.layers.Dense(activation="softmax", units=3, kernel_initializer='uniform'))

In [ ]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
classifier.fit(X_train_norm,y_train_one_hot,batch_size=5,epochs=90)

Epoch 1/90
24/24 [==============================] - 1s 2ms/step - loss: 1.0988 - accuracy: 0.2833
Epoch 2/90
24/24 [==============================] - 0s 2ms/step - loss: 1.0986 - accuracy: 0.3417
Epoch 3/90
24/24 [==============================] - 0s 2ms/step - loss: 1.0979 - accuracy: 0.3083
Epoch 4/90
24/24 [==============================] - 0s 2ms/step - loss: 1.0962 - accuracy: 0.5917
Epoch 5/90
24/24 [==============================] - 0s 2ms/step - loss: 1.0913 - accuracy: 0.6333
Epoch 6/90
24/24 [==============================] - 0s 2ms/step - loss: 1.0825 - accuracy: 0.6250
Epoch 7/90
24/24 [==============================] - 0s 2ms/step - loss: 1.0692 - accuracy: 0.6000
Epoch 8/90
24/24 [==============================] - 0s 2ms/step - loss: 1.0456 - accuracy: 0.6750
Epoch 9/90
24/24 [==============================] - 0s 2ms/step - loss: 1.0128 - accuracy: 0.6583
Epoch 10/90
24/24 [==============================] - 0s 2ms/step - loss: 0.9686 - accuracy: 0.6583
Epoch 11/90
24/24 [